In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
#session start
spark_session = SparkSession.builder\
    .master("spark://192.168.2.235:7077") \
     .appName("group11_scalability_testing_4")\
     .config("spark.dynamicAllocation.enabled", True)\
     .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
     .config("spark.shuffle.service.enabled", True)\
     .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
     .config("spark.cores.max", 4)\
     .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

nr_entries = 10000
#data collection from HDFS
song_hotness = spark_context.textFile('hdfs://192.168.2.235:50000/data/splits/song_hotttnesss.csv')
release_year = spark_context.textFile('hdfs://192.168.2.235:50000/data/splits/year.csv')
namest = spark_context.textFile('hdfs://192.168.2.235:50000/data/splits/artist_name.csv')
hotnesst = spark_context.textFile('hdfs://192.168.2.235:50000/data/splits/song_hotttnesss.csv')
loudness = spark_context.textFile('hdfs://192.168.2.235:50000/data/splits/loudness.csv')
hotness = spark_context.textFile('hdfs://192.168.2.235:50000/data/splits/song_hotttnesss.csv')
names_q3 = spark_context.textFile('hdfs://192.168.2.235:50000/data/splits/artist_name.csv')

#question 1 calculations 
hotnes = hotnesst.take(nr_entries)
name = namest.take(nr_entries)

hottness = pd.DataFrame(hotnes)
names = pd.DataFrame(name)

df = pd.concat([hottness, names], axis=1)
df.columns = ['value', 'artist']
df['value'] = pd.to_numeric(df['value'], errors='coerce')
df = df.dropna(subset=['value'])
dfs = df.sort_values(by= 'value', ascending=False)

#question 2 calculations
hotness = song_hotness.take(nr_entries)
year = release_year.take(nr_entries)

df_hotness = pd.DataFrame(hotness)
df_year = pd.DataFrame(year)

df = pd.concat([df_hotness,df_year], axis = 1)
df.columns = ['hotness', 'year']
df['year'] = pd.to_numeric(df['year'], errors='coerce')
df['hotness']= pd.to_numeric(df['hotness'], errors='coerce')
filtered_df = df[(df['year']>0) & df['hotness']>0]

average_per_year = filtered_df.select_dtypes(include= np.number).groupby("year").mean()

top5_years=average_per_year.sort_values(by="hotness", ascending= False).head()

#question 3 calculations
mapper = names_q3.map(lambda word: (word, 1))
name_count = mapper.reduceByKey(lambda x,y : x+y).take(nr_entries)
name_count_sorted = sorted(name_count, key=lambda x:x[1], reverse=True)
title = "Most occurring artists: \n"
for i in range(10):
    title += (f"{i+1}: {name_count_sorted[i][0]}, {name_count_sorted[i][1]}" + "\n")
spark_session.stop()